# Estudi Gaussian Mixture

## Lectura fitxer students

In [ ]:
import pandas as pd

In [ ]:
df_students = pd.read_csv("C:/Users/MLi/Documents/UAB/AC/Projecte/ACproject-07/datasets/Data Carrard.csv")
df_students = df_students.drop('id', axis = 1)

In [ ]:
df_categorical = df_students[['sex', 'year', 'glang', 'part', 'job', 'stud_h', 'health', 'psyt']]
df_numerical = df_students[['age', 'jspe', 'qcae_cog', 'qcae_aff', 'amsp', 'erec_mean', 'cesd', 'stai_t', 'mbi_ex', 'mbi_cy', 'mbi_ea']]

In [ ]:
def sampleig_dataset(dataset, frac):
    dataset = dataset.sample(frac=frac).reset_index(drop=True)
    return dataset

In [ ]:
frac_train = 0.7
frac_test = 0.3

X_train = sampleig_dataset(df_students, frac_train)
X_test = sampleig_dataset(df_students, frac_test)

## Aplicació model

In [ ]:
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
n_grups = 3
model_gm = GaussianMixture(n_components=n_grups)

In [ ]:
def generacio_clusters(model, X, prob=False):
    model.fit(X)

    # prediccio del model sobre tots els pixels de la imatge (omplir variable labels)
    if not prob:
        labels = model.predict(X)
    else: # predicció probabilística de GaussianMixture
        labels = model.predict_proba(X)
    
    # agafar els clusters segons si es k-means o gmm (estan en atributs diferents)
    # if isinstance(model, KMeans):
    #     clusters = model.cluster_centers_
    # else: # GaussianMixture
    clusters = model.means_
        
    return clusters, labels

In [ ]:
clusters_gm, labels_gm = generacio_clusters(model_gm, X_train)



In [ ]:
sns.scatterplot(x=clusters[:, 0], y=clusters[:, 1], hue=labels,
                    legend=False, alpha=0.5,
                    palette=sns.color_palette('magma', n_colors=len(np.unique(labels))))
plt.title("Clusters")

## Avaluació model

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer

### Elbow Visualizer

In [ ]:
plt.figure(figsize=(20,20))
visualizer = KElbowVisualizer(model_gm, k=n_grups)
visualizer.fit(X_train)   

### Silouhette Visualizer

In [ ]:
plt.figure(figsize=(20,20))
results = []
i=0
k_tries = range(2, 10)
for k in k_tries:
    i+=1
    plt.subplot(len(k_tries),1, i)
    model = GaussianMixture(n_clusters=k, n_init='auto')
    visualizer = SilhouetteVisualizer(model, colors='yellowbrick')

    # X, y = get_data('circles')
    visualizer.fit(X)        # Fit the data to the visualizer
    results.append((k, model.inertia_, visualizer.silhouette_score_))
    print(results[-1])
